In [1]:
spark

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
.appName("DTHandling")\
.getOrCreate()

25/09/01 15:31:16 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
data = [
    (1, "John Doe", "Bangalore", "2023-01-15", "152.75", "True"),
    (2, "Jane Smith", "Delhi", "2023-05-20", "89.50", "False"),
    (3, "Robert Brown", "Mumbai", "InvalidDate", "200.00", "True"),
    (4, "Linda White", "Kolkata", "2023-02-29", None, "yes"),   # Feb 29 invalid in 2023
    (5, "Mike Green", "Chennai", "2023-08-10", "NaN", "1"),     # NaN needs handling
    (6, "Sarah Blue", "Hyderabad", "InvalidDate", "300.40", "No")
]

cols = ["id","name","city","date","amount","is_active"]

df = spark.createDataFrame(data, cols)

df.show()

+---+------------+---------+-----------+------+---------+
| id|        name|     city|       date|amount|is_active|
+---+------------+---------+-----------+------+---------+
|  1|    John Doe|Bangalore| 2023-01-15|152.75|     True|
|  2|  Jane Smith|    Delhi| 2023-05-20| 89.50|    False|
|  3|Robert Brown|   Mumbai|InvalidDate|200.00|     True|
|  4| Linda White|  Kolkata| 2023-02-29|  NULL|      yes|
|  5|  Mike Green|  Chennai| 2023-08-10|   NaN|        1|
|  6|  Sarah Blue|Hyderabad|InvalidDate|300.40|       No|
+---+------------+---------+-----------+------+---------+



In [4]:
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- city: string (nullable = true)
 |-- date: string (nullable = true)
 |-- amount: string (nullable = true)
 |-- is_active: string (nullable = true)



In [5]:
from pyspark.sql.types import IntegerType, StringType, BooleanType, StructField, StructType
from pyspark.sql.functions import col

In [6]:
df = df.withColumn('id', col('id').cast(IntegerType()))

In [7]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- city: string (nullable = true)
 |-- date: string (nullable = true)
 |-- amount: string (nullable = true)
 |-- is_active: string (nullable = true)



In [8]:
df = df.withColumn('amount', col('amount').cast('float')).fillna({'amount':0})

In [9]:
df.show()

+---+------------+---------+-----------+------+---------+
| id|        name|     city|       date|amount|is_active|
+---+------------+---------+-----------+------+---------+
|  1|    John Doe|Bangalore| 2023-01-15|152.75|     True|
|  2|  Jane Smith|    Delhi| 2023-05-20|  89.5|    False|
|  3|Robert Brown|   Mumbai|InvalidDate| 200.0|     True|
|  4| Linda White|  Kolkata| 2023-02-29|   0.0|      yes|
|  5|  Mike Green|  Chennai| 2023-08-10|   0.0|        1|
|  6|  Sarah Blue|Hyderabad|InvalidDate| 300.4|       No|
+---+------------+---------+-----------+------+---------+



In [10]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- city: string (nullable = true)
 |-- date: string (nullable = true)
 |-- amount: float (nullable = false)
 |-- is_active: string (nullable = true)



In [11]:
df = df.withColumn('date', col('date').cast('date')) #By default "YYYY-MM-DD HH-MM-SS" is the format

In [12]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- city: string (nullable = true)
 |-- date: date (nullable = true)
 |-- amount: float (nullable = false)
 |-- is_active: string (nullable = true)



In [13]:
df.show()

+---+------------+---------+----------+------+---------+
| id|        name|     city|      date|amount|is_active|
+---+------------+---------+----------+------+---------+
|  1|    John Doe|Bangalore|2023-01-15|152.75|     True|
|  2|  Jane Smith|    Delhi|2023-05-20|  89.5|    False|
|  3|Robert Brown|   Mumbai|      NULL| 200.0|     True|
|  4| Linda White|  Kolkata|      NULL|   0.0|      yes|
|  5|  Mike Green|  Chennai|2023-08-10|   0.0|        1|
|  6|  Sarah Blue|Hyderabad|      NULL| 300.4|       No|
+---+------------+---------+----------+------+---------+



In [15]:
from pyspark.sql.functions import to_timestamp, year, month, dayofmonth, hour, minute #datediff

df_split = df\
.withColumn('year', year(df.date))\
.withColumn('day', dayofmonth(df.date))\
.withColumn('month', month(df.date))

In [16]:
df_split.show()

+---+------------+---------+----------+------+---------+----+----+-----+
| id|        name|     city|      date|amount|is_active|year| day|month|
+---+------------+---------+----------+------+---------+----+----+-----+
|  1|    John Doe|Bangalore|2023-01-15|152.75|     True|2023|  15|    1|
|  2|  Jane Smith|    Delhi|2023-05-20|  89.5|    False|2023|  20|    5|
|  3|Robert Brown|   Mumbai|      NULL| 200.0|     True|NULL|NULL| NULL|
|  4| Linda White|  Kolkata|      NULL|   0.0|      yes|NULL|NULL| NULL|
|  5|  Mike Green|  Chennai|2023-08-10|   0.0|        1|2023|  10|    8|
|  6|  Sarah Blue|Hyderabad|      NULL| 300.4|       No|NULL|NULL| NULL|
+---+------------+---------+----------+------+---------+----+----+-----+



In [ ]:
spark.stop()